In [1]:
import pandas as pd

In [2]:
file_path = "test/Data/AAPL.csv"
data = pd.read_csv(file_path, index_col=0)

In [3]:
x = data.iloc[:,0:6]
y = data.iloc[:,7]

# feature names should not have empty spaces in them
features = ['Adj_Close', 'Close', 'High', 'Low', 'Open', 'Volume']

x.columns = features

In [4]:
x_train,x_test = x.iloc[:5000,:], x.iloc[5000:,:]
y_train,y_test = y[:5000],y[5000:]

## XGB

In [5]:
import xgboost as xgb

In [6]:
model_xgb = xgb.XGBRegressor()
bst=model_xgb.fit(x_train, y_train)

# dumping the model detils to a file
model_xgb._Booster.dump_model("./test/Models/xgb_model.txt")

## Volta XGB

In [7]:
import voltatrees.volta_XGBM as vxgb

In [8]:
model = vxgb.Model("./test/Models/xgb_model.txt", features)

In [9]:
model.compile()

In [10]:
our_predictions = model.predict(x_test)

In [11]:
our_predictions

array([ 0.03996586, -0.28144285,  0.49001607, ...,  0.7893066 ,
        0.71400942,  0.75283789])

In [12]:
%%timeit -n 2 -r 3

model.predict(x, n_jobs=3)

7.32 ms ± 1.15 ms per loop (mean ± std. dev. of 3 runs, 2 loops each)


## Treelite

In [13]:
import treelite

In [14]:
model = treelite.Model.from_xgboost(model_xgb._Booster)

In [15]:
model.export_lib(toolchain="gcc", libpath='./mymodel.so', verbose=True)

[14:45:58] ../src/compiler/ast/split.cc:24: Parallel compilation disabled; all member trees will be dumped to a single source file. This may increase compilation time and memory usage.
[14:45:58] /home/aswin/anaconda3/envs/ml/lib/python3.9/site-packages/treelite/contrib/__init__.py:267: WARNING: some of the source files are long. Expect long build time. You may want to adjust the parameter parallel_comp.

[14:45:58] /home/aswin/anaconda3/envs/ml/lib/python3.9/site-packages/treelite/contrib/util.py:105: Compiling sources files in directory ./tmpgrjl71rp into object files (*.o)...
[14:46:28] /home/aswin/anaconda3/envs/ml/lib/python3.9/site-packages/treelite/contrib/util.py:135: Generating dynamic shared library ./tmpgrjl71rp/predictor.so...
[14:46:28] /home/aswin/anaconda3/envs/ml/lib/python3.9/site-packages/treelite/contrib/__init__.py:282: Generated shared library in 30.20 seconds


In [16]:
import treelite_runtime     # runtime module
predictor = treelite_runtime.Predictor('./mymodel.so', verbose=True)

[14:46:28] /home/aswin/.local/lib/python3.9/site-packages/treelite_runtime/predictor.py:159: Dynamic shared library /home/aswin/volta-trees/mymodel.so has been successfully loaded into memory


In [17]:
dmat = treelite_runtime.DMatrix(x_test)

In [18]:
out_pred = predictor.predict(dmat)

In [19]:
out_pred

array([ 0.03996587, -0.28144288,  0.4900161 , ...,  0.7893065 ,
        0.7140093 ,  0.7528378 ], dtype=float32)

In [20]:
dmat2 = treelite_runtime.DMatrix(x)

In [21]:
%%timeit -n 2 -r 3

predictor.predict(dmat2)

14.3 ms ± 3.61 ms per loop (mean ± std. dev. of 3 runs, 2 loops each)
